In [ ]:
import os
import sys

PROJECT_ROOT = "/home/pablo/Documents/datathon2025-smadex"

if PROJECT_ROOT not in sys.path:
	sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("Contenido de ./data:", os.listdir(os.path.join(PROJECT_ROOT, "data")))

PROJECT_ROOT: /home/pablo/Documents/datathon2025-smadex


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '/home/pablo/Documents/datathon2025-smadex\\data'

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd



PROJECT_ROOT = os.getcwd() 

if PROJECT_ROOT not in sys.path:
	sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("Contenido de ./data:", os.listdir(os.path.join(PROJECT_ROOT, "data")))

PROJECT_ROOT: c:\Users\adria.flores\Documents\Projects\hacks\datathon2025-smadex
Contenido de ./data: ['sample_submission.csv', 'split', 'test', 'train']


In [2]:
import dask
import dask.dataframe as dd
import pandas as pd
import json
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from typing import Dict, List
dask.config.set({"dataframe.convert-string": False})

In [ ]:

DATASET_PATH = "./data/train"
EMBEDDINGS_MAPPING_FILE = "embeddings_mappings.json"
SCALER_FILE = "scaler.joblib"

CATEGORICAL_FEATURES = ['advertiser_bundle', 'advertiser_category', 'advertiser_subcategory', 'advertiser_bottom_taxonomy_level', 'country', 'dev_make', 'dev_model', 'dev_os', 'dev_osv', 'release_date']
NUMERICAL_FEATURES = ['release_msrp', 'weekday', 'avg_act_days', 'weekend_ratio', 'weeks_since_first_seen', 'wifi_ratio', 'hours_since_last_buy', 'hours_since_last_ins', 'hour_sin', 'hour_cos', 'first_request_ts_bundle_hours_ago', 'last_buy_ts_bundle_hours_ago', 'last_buy_ts_category_hours_ago', 'user_actions_bundles_action_last_timestamp_hours_ago', 'iap_revenue_usd_bundle_agg', 'num_buys_bundle_agg', 'rev_by_adv_agg', 'rwd_prank_agg', 'whale_users_bundle_num_buys_prank_agg', 'whale_users_bundle_revenue_prank_agg', 'whale_users_bundle_total_num_buys_agg', 'whale_users_bundle_total_revenue_agg', 'avg_daily_sessions_agg', 'avg_duration_agg', 'bcat_bottom_taxonomy_agg', 'ctr_agg', 'bundles_cat_bottom_taxonomy_agg']
ALL_FEATURES = CATEGORICAL_FEATURES + NUMERICAL_FEATURES + [f"{col}_is_missing" for col in NUMERICAL_FEATURES]

TARGET = "iap_revenue_d7"
TARGET_BIN = "buy_d7_binary"        # 0/1 compra
TARGET_LOG = "iap_revenue_d7_log"  

def load_embeddings_mapping():
	with open(EMBEDDINGS_MAPPING_FILE, "r") as f:
		mappings = json.load(f)
	return mappings

EMBEDDING_MAPPINGS = load_embeddings_mapping()
COLS_TO_READ = [
			 'iap_revenue_d7', 
			 'advertiser_bundle', 
			 'advertiser_category', 
			 'advertiser_subcategory', 
			 'advertiser_bottom_taxonomy_level', 
			 'country', 
			 'dev_make', 
			 'dev_model', 
			 'dev_os', 
			 'dev_osv', 
			 'hour', 
			 'release_date', 
			 'release_msrp', 
			 'weekday', 
			 'avg_act_days', 
			 'avg_daily_sessions', 
			 'avg_duration', 
			 'bcat_bottom_taxonomy', 
			 'bundles_cat_bottom_taxonomy',  
			 'ctr',  
			 'first_request_ts_bundle', 
			 'iap_revenue_usd_bundle', 
			 'last_buy', 
			 'last_buy_ts_bundle', 
			 'last_buy_ts_category', 
			 'last_ins', 
			 'user_actions_bundles_action_last_timestamp', 
			 'num_buys_bundle', 
			 'rev_by_adv', 
			 'rwd_prank', 
			 'weekend_ratio', 
			 'weeks_since_first_seen', 
			 'wifi_ratio', 
			 'whale_users_bundle_num_buys_prank', 
			 'whale_users_bundle_revenue_prank', 
			 'whale_users_bundle_total_num_buys', 
			 'whale_users_bundle_total_revenue']

loaded_scaler = joblib.load(SCALER_FILE)

In [ ]:
def process_partition(df):
	df = transform_variables(df)
	df = scale_numerical_features(df, loaded_scaler)
	df = impute_missings(df)
	return df

def hours_since_now_from_list(tuples_list, now_ts):
	# Check if the input is a list
	if isinstance(tuples_list, list) and len(tuples_list) > 0:
		# Extract all numeric timestamps from the tuples
		timestamps = []
		for t in tuples_list:
			if isinstance(t, tuple) and len(t) == 2:
				ts = t[1]
				if ts is not None and not isinstance(ts, list):
					timestamps.append(ts)
		if len(timestamps) > 0:
			# Use the largest timestamp (closest to now)
			max_ts = max(timestamps)
			return (now_ts - max_ts) / 3600  # seconds → hours
	return np.nan

def extract_numbers(tuple_list):
	"""Extract only the numeric part from a list of (id, value) tuples."""
	if isinstance(tuple_list, list):
		return [t[1] for t in tuple_list if isinstance(t, tuple) and len(t) >= 2]
	return []

def aggregate(values, mode):
	"""Apply either sum or mean depending on mode."""
	if not values:
		return np.nan
	if mode == "sum":
		return sum(values)
	return sum(values) / len(values)

def transform_variables(df):
	 
	try:

			now = pd.Timestamp.now()
			# Define reasonable bounds for Unix timestamps (seconds)
			min_ts = 0                  # 1970-01-01
			max_ts = 4102444800         # 2100-01-01 in Unix seconds

			# Replace invalid timestamps with NaN
			df["last_buy_safe"] = df["last_buy"].where(
				df["last_buy"].between(min_ts, max_ts), np.nan
			)
			df["last_ins_safe"] = df["last_ins"].where(
				df["last_ins"].between(min_ts, max_ts), np.nan
			)

			# Convert safe Unix timestamps to datetime
			df["last_buy_dt"] = pd.to_datetime(df["last_buy_safe"], unit="s")
			df["last_ins_dt"] = pd.to_datetime(df["last_ins_safe"], unit="s")

			# Compute hours ago
			df["hours_since_last_buy"] = (now - df["last_buy_dt"]).dt.total_seconds() / 3600
			df["hours_since_last_ins"] = (now - df["last_ins_dt"]).dt.total_seconds() / 3600

			# Drop the original Unix timestamp columns
			df = df.drop(columns=["last_buy", "last_ins", "last_buy_safe", "last_ins_dt", "last_buy_dt", "last_ins_safe"])




			# Convert 'hour' from string to integer
			df['hour'] = df['hour'].astype(int)

			# Convert hour to radians (full circle = 24 hours)
			radians = df['hour'] * (2 * np.pi / 24)

			# Create two new columns: sin_hour and cos_hour
			df['hour_sin'] = np.sin(radians)
			df['hour_cos'] = np.cos(radians)

			# Drop the original 'hour' column
			df.drop(columns=['hour'], inplace=True)




			# Ensure `now_ts` is a Unix timestamp
			now_ts = int(pd.Timestamp.now().timestamp())

			# Apply to your 4 timestamp columns
			ts_cols = [
				"first_request_ts_bundle",
				"last_buy_ts_bundle",
				"last_buy_ts_category",
				"user_actions_bundles_action_last_timestamp"
			]

			for col in ts_cols:
				new_col = col + "_hours_ago"
				df[new_col] = df[col].apply(lambda x: hours_since_now_from_list(x, now_ts))

			# Drop the original tuple columns if you want
			df.drop(columns=ts_cols, inplace=True)




			rules = {
				"iap_revenue_usd_bundle": "sum",
				"num_buys_bundle": "sum",
				"rev_by_adv": "sum",
				"rwd_prank": "mean",
				"whale_users_bundle_num_buys_prank": "mean",
				"whale_users_bundle_revenue_prank": "mean",
				"whale_users_bundle_total_num_buys": "sum",
				"whale_users_bundle_total_revenue": "sum",
				"avg_daily_sessions": "mean",
				"avg_duration": "mean",
				"bcat_bottom_taxonomy": "mean",
				"ctr": "sum",
				"bundles_cat_bottom_taxonomy": "sum",
			}

			for col, mode in rules.items():
				new_col = col + "_agg"
				df[new_col] = df[col].apply(
					lambda lst: aggregate(extract_numbers(lst), mode)
				)

			df.drop(columns=list(rules.keys()), inplace=True)

			# If the df contains the target column, transform it with the logarithm
			if TARGET in df.columns:
				# 1) binario de compra
				df[TARGET_BIN] = (df[TARGET] > 0).astype(float)

				# 2) log1p del importe (clip por si acaso)
				df[TARGET_LOG] = np.log1p(df[TARGET].clip(lower=0))

	# Remove all the variables and create the new ones by setting them to the default values
	except Exception:
		# If something failed, remove any partially created/intermediate columns
		ts_cols = [
			"first_request_ts_bundle",
			"last_buy_ts_bundle",
			"last_buy_ts_category",
			"user_actions_bundles_action_last_timestamp",
		]

		rules_keys = [
			"iap_revenue_usd_bundle",
			"num_buys_bundle",
			"rev_by_adv",
			"rwd_prank",
			"whale_users_bundle_num_buys_prank",
			"whale_users_bundle_revenue_prank",
			"whale_users_bundle_total_num_buys",
			"whale_users_bundle_total_revenue",
			"avg_daily_sessions",
			"avg_duration",
			"bcat_bottom_taxonomy",
			"ctr",
			"bundles_cat_bottom_taxonomy",
		]

		intermediate_cols = (
			[
				"last_buy",
				"last_ins",
				"last_buy_safe",
				"last_ins_safe",
				"last_buy_dt",
				"last_ins_dt",
				"hour",
				"hours_since_last_buy",
				"hours_since_last_ins",
				"hour_sin",
				"hour_cos",
			]
			+ [c + "_hours_ago" for c in ts_cols]
			+ [k + "_agg" for k in rules_keys]
		)

		# Drop any of those columns if they exist (silent if not)
		df.drop(columns=[c for c in intermediate_cols if c in df.columns], inplace=True, errors="ignore")

		# Create all expected features with default (NaN) values so subsequent steps won't fail
		for col in ALL_FEATURES:
			if col not in df.columns:
				df[col] = np.nan

		# Ensure target exists (set to NaN)
		if TARGET not in df.columns:
			df[TARGET] = np.nan

		# NUEVO: asegurarnos de que también existen los targets derivados
		if TARGET_BIN not in df.columns:
			df[TARGET_BIN] = np.nan

		if TARGET_LOG not in df.columns:
			df[TARGET_LOG] = np.nan

	
	return df

def impute_missings(df):

	for col in CATEGORICAL_FEATURES:
		df[col] = df[col].fillna("<MISSING>")

		# convert to embeddings indices (if category not seen before, assign index 0)
		mapping = EMBEDDING_MAPPINGS[col]
		df[col] = df[col].map(lambda x: mapping.get(x, 0))

	for col in NUMERICAL_FEATURES:
		# 1. Create missing indicator
		df[f"{col}_is_missing"] = df[col].isna().astype(float)

		# 2. Impute missings with 0
		df[col] = df[col].fillna(0).astype(float)

	return df

def scale_numerical_features(df, scaler):
	df[NUMERICAL_FEATURES] = scaler.transform(df[NUMERICAL_FEATURES])
	return df

def train_scaler(df):
	scaler = StandardScaler()
	scaler.fit(df[NUMERICAL_FEATURES])
	joblib.dump(scaler, SCALER_FILE)
	print("Scaler saved.")

def generate_embeddings_mapping(pdf):
	# Use after transform_variables but BEFORE imputing missings
	
	mappings = {}
	for feature in CATEGORICAL_FEATURES:
		unique_values = ["<MISSING>"] + pdf[feature].dropna().unique().tolist()
		embeddings = {val: idx for idx, val in enumerate(unique_values)}
		mappings[feature] = embeddings
	
	# Save mappings to disk
	with open(EMBEDDINGS_MAPPING_FILE, "w") as f:
		json.dump(mappings, f)


In [ ]:
VERSION = "v1_f"
EPOCHS = 5
BATCH_SIZE = 8192
LEARNING_RATE = 0.005
CHECKPOINT_PATH = "model_checkpoint_" + VERSION
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# AMP = False

num_cont_features = len(NUMERICAL_FEATURES) + len(NUMERICAL_FEATURES)  # numéricas + _is_missing

cat_cardinalities = {
	feat: len(EMBEDDING_MAPPINGS[feat])
	for feat in CATEGORICAL_FEATURES
}

class DeepEmbeddingMLP(nn.Module):
	def __init__(
		self,
		cat_cardinalities: Dict[str, int],
		cat_features: List[str],
		num_cont_features: int,          # numéricas + _is_missing
		embedding_dim: int = 32,
		hidden_dims: List[int] = [512, 256, 128, 64],
		dropout: float = 0.2,
	):
		super().__init__()
		self.cat_features = cat_features
		self.num_cat = len(cat_features)
		self.num_cont_features = num_cont_features

		# Embeddings por feature categórica
		self.embeddings = nn.ModuleDict({
			feat: nn.Embedding(num_embeddings=cardinality, embedding_dim=embedding_dim)
			for feat, cardinality in cat_cardinalities.items()
		})

		total_cat_dim = embedding_dim * self.num_cat
		input_dim = total_cat_dim + self.num_cont_features

		layers = []
		prev_dim = input_dim
		for h in hidden_dims:
			layers.append(nn.Linear(prev_dim, h))
			layers.append(nn.ReLU())
			layers.append(nn.Dropout(dropout))
			prev_dim = h

		# salida escalar (logits para clasificación o valor para regresión)
		layers.append(nn.Linear(prev_dim, 1))
		self.mlp = nn.Sequential(*layers)

	def forward(self, x: torch.Tensor) -> torch.Tensor:
		"""
		x: [batch, num_cat + num_cont_features]
		columnas en el orden:
		[CATEGORICAL_FEATURES..., NUMERICAL_FEATURES..., NUMERICAL_IS_MISSING...]
		"""
		# separar cat / num
		x_cat_ids = x[:, :self.num_cat].long()  # [B, num_cat]
		x_cont = x[:, self.num_cat:].float()    # [B, num_cont]

		# embeddings categóricas
		emb_list = []
		for i, feat in enumerate(self.cat_features):
			emb = self.embeddings[feat](x_cat_ids[:, i])  # [B, embedding_dim]
			emb_list.append(emb)

		if emb_list:
			x_cat_emb = torch.cat(emb_list, dim=1)        # [B, embedding_dim * num_cat]
			x_in = torch.cat([x_cont, x_cat_emb], dim=1)  # [B, input_dim]
		else:
			x_in = x_cont

		out = self.mlp(x_in)          # [B, 1]
		return out.squeeze(-1)

In [ ]:
def make_features_tensor(pdf: pd.DataFrame) -> torch.Tensor:
	return torch.tensor(
		pdf[
			CATEGORICAL_FEATURES
			+ NUMERICAL_FEATURES
			+ [f"{col}_is_missing" for col in NUMERICAL_FEATURES]
		].values,
		dtype=torch.float32,
	)

In [ ]:
def train_stage(
	stage_name: str,
	ddf_train,
	ddf_val,
	target_col: str,
	only_buyers: bool,
	criterion,
	checkpoint_prefix: str,
	epochs: int = EPOCHS,
	amp: bool = False,
	init_model: nn.Module | None = None,
):
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

	# --- VALIDACIÓN ---
	val_pdf = ddf_val.compute()
	val_pdf = process_partition(val_pdf)
	if only_buyers:
		val_pdf = val_pdf[val_pdf[TARGET_BIN] == 1].copy()

	val_x = make_features_tensor(val_pdf)
	val_y = torch.tensor(val_pdf[target_col].values, dtype=torch.float32)

	val_dataset = TensorDataset(val_x, val_y)
	val_loader = DataLoader(
		dataset=val_dataset,
		batch_size=BATCH_SIZE,
		num_workers=0,
		pin_memory=True,
		shuffle=False,
	)

	# --- MODELO ---
	if init_model is not None:
		model = init_model
	else:
		model = DeepEmbeddingMLP(
			cat_cardinalities=cat_cardinalities,
			cat_features=CATEGORICAL_FEATURES,
			num_cont_features=num_cont_features,
			embedding_dim=32,
			hidden_dims=[512, 256, 128, 64],
			dropout=0.2,
		).to(device)

	optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
	scaler = torch.amp.GradScaler(device="cuda", enabled=amp)

	for epoch in range(epochs):
		print(f"\n[{stage_name}] --- Epoch {epoch + 1}/{epochs} ---")
		model.train()

		partition_iterator = ddf_train.to_delayed()
		for partition_idx, partition in enumerate(partition_iterator):
			train_loss_sum = 0.0

			pdf = partition.compute()
			if pdf.empty:
				print(f"  [{stage_name}] Skipping empty partition {partition_idx+1}")
				continue

			pdf = process_partition(pdf)
			if only_buyers:
				pdf = pdf[pdf[TARGET_BIN] == 1].copy()
				if pdf.empty:
					print(f"  [{stage_name}] Partition {partition_idx+1} has no buyers, skipping")
					continue

			x = make_features_tensor(pdf)
			y = torch.tensor(pdf[target_col].values, dtype=torch.float32)

			dataset = TensorDataset(x, y)
			loader = DataLoader(
				dataset=dataset,
				batch_size=BATCH_SIZE,
				num_workers=0,
				pin_memory=True,
				shuffle=True,
			)

			for batch_idx, (data, target) in enumerate(loader):
				data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)

				optimizer.zero_grad(set_to_none=True)
				with torch.amp.autocast(device_type="cuda", enabled=amp):
					out = model(data)         # [B]
					loss = criterion(out, target)

				if amp:
					scaler.scale(loss).backward()
					scaler.step(optimizer)
					scaler.update()
				else:
					loss.backward()
					optimizer.step()

				train_loss_sum += loss.item()

			print(
				f"  [{stage_name}] Epoch {epoch+1} | Partition {partition_idx+1} "
				f"| Loss: {train_loss_sum / len(loader):.4f}"
			)

		# VALIDACIÓN
		val_loss_sum = 0.0
		model.eval()
		with torch.no_grad():
			for data, target in val_loader:
				data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)

				with torch.amp.autocast(device_type="cuda", enabled=amp):
					out = model(data)
					loss = criterion(out, target)

				val_loss_sum += loss.item()

		print(f"[{stage_name}] Epoch {epoch + 1} | Val loss: {val_loss_sum / len(val_loader):.4f}")
		torch.save(model.state_dict(), f"{checkpoint_prefix}_e{epoch+1}.pth")
		print(f"[{stage_name}] Saved: {checkpoint_prefix}_e{epoch+1}.pth")

	print(f"\n[{stage_name}] --- Training Finished ---")

In [ ]:
def build_regressor_from_classifier_ckpt(ckpt_path: str) -> nn.Module:
	# Crear el modelo con la MISMA arquitectura
	model = DeepEmbeddingMLP(
		cat_cardinalities=cat_cardinalities,
		cat_features=CATEGORICAL_FEATURES,
		num_cont_features=num_cont_features,
		embedding_dim=32,
		hidden_dims=[512, 256, 128, 64],
		dropout=0.2,
	).to(device)

	state = torch.load(ckpt_path, map_location=device)
	model.load_state_dict(state)

	# OPCIONAL: re-inicializar SOLO la última capa lineal (la "head")
	# para que no arrastre el sesgo de clasificación
	last_layer = model.mlp[-1]
	if isinstance(last_layer, nn.Linear):
		nn.init.xavier_uniform_(last_layer.weight)
		nn.init.zeros_(last_layer.bias)

	return model

In [ ]:
def train_two_stage():
	AMP = False
	if not torch.cuda.is_available() or not torch.amp.autocast_mode.is_autocast_available("cuda"):
		AMP = False

	train_filters = [("datetime", "<", "2025-10-06-23-00")]
	val_filters = [("datetime", ">=", "2025-10-06-23-00")]

	ddf_train = dd.read_parquet(
		DATASET_PATH,
		engine="pyarrow",
		columns=COLS_TO_READ,
		filters=train_filters,
	)

	ddf_val = dd.read_parquet(
		DATASET_PATH,
		engine="pyarrow",
		columns=COLS_TO_READ,
		filters=val_filters,
	)

	# 1) (Opcional) entrenar el clasificador aquí si no lo tienes ya
	#    Si ya lo tienes, te lo saltas.

	# 2) Regresor inicializado desde el clasificador
	cls_ckpt_path = "model_checkpoint_v1_f_cls_e1.pth"  # el que tengas

	reg_model = DeepEmbeddingMLP(
	cat_cardinalities=cat_cardinalities,
	cat_features=CATEGORICAL_FEATURES,
	num_cont_features=num_cont_features,  # = 2 * len(NUMERICAL_FEATURES)
	embedding_dim=64,                     # <-- IGUAL que en el clasificador
	hidden_dims=[512, 256, 128, 64],      # <-- IGUAL que en el clasificador
	dropout=0.2,
	).to(device)
	state = torch.load("model_checkpoint_v1_f_cls_e1.pth", map_location=device)
	reg_model.load_state_dict(state) 
	last_layer = reg_model.mlp[-1]
	if isinstance(last_layer, nn.Linear):
		nn.init.xavier_uniform_(last_layer.weight)
		nn.init.zeros_(last_layer.bias)

	train_stage(
		stage_name="REG",
		ddf_train=ddf_train,
		ddf_val=ddf_val,
		target_col=TARGET_LOG,          # "iap_revenue_d7_log"
		only_buyers=True,
		criterion=nn.MSELoss(),
		checkpoint_prefix=f"{CHECKPOINT_PATH}_reg",
		epochs=EPOCHS,
		amp=AMP,
		init_model=reg_model,
	)

In [ ]:
if __name__ == "__main__":
	train_two_stage()


[REG] --- Epoch 1/5 ---
  [REG] Epoch 1 | Partition 1 | Loss: 12.1108
  [REG] Epoch 1 | Partition 2 | Loss: 5.7939
  [REG] Epoch 1 | Partition 3 | Loss: 4.7506
  [REG] Epoch 1 | Partition 4 | Loss: 3.8539
  [REG] Epoch 1 | Partition 5 | Loss: 2.9583
  [REG] Epoch 1 | Partition 6 | Loss: 2.9573
  [REG] Epoch 1 | Partition 7 | Loss: 2.8712
  [REG] Epoch 1 | Partition 8 | Loss: 2.6486
  [REG] Epoch 1 | Partition 9 | Loss: 2.6269
  [REG] Epoch 1 | Partition 10 | Loss: 2.5844
  [REG] Epoch 1 | Partition 11 | Loss: 2.3911
  [REG] Epoch 1 | Partition 12 | Loss: 2.3904
  [REG] Epoch 1 | Partition 13 | Loss: 2.3063
  [REG] Epoch 1 | Partition 14 | Loss: 2.0670
  [REG] Epoch 1 | Partition 15 | Loss: 1.8276
  [REG] Epoch 1 | Partition 16 | Loss: 1.7829
  [REG] Epoch 1 | Partition 17 | Loss: 1.6787
  [REG] Epoch 1 | Partition 18 | Loss: 1.6041
  [REG] Epoch 1 | Partition 19 | Loss: 1.5904
  [REG] Epoch 1 | Partition 20 | Loss: 1.4943
  [REG] Epoch 1 | Partition 21 | Loss: 1.4504
  [REG] Epoch 1 |

KeyboardInterrupt: 

In [ ]:
def fine_tune_with_val(best_ckpt_path: str):
	AMP = False
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

	# 1) Cargar train y val y concatenarlos
	train_filters = [("datetime", "<", "2025-10-06-23-00")]
	val_filters = [("datetime", ">=", "2025-10-06-23-00")]

	ddf_train = dd.read_parquet(
		DATASET_PATH,
		engine="pyarrow",
		columns=COLS_TO_READ,
		filters=train_filters,
	)
	ddf_val = dd.read_parquet(
		DATASET_PATH,
		engine="pyarrow",
		columns=COLS_TO_READ,
		filters=val_filters,
	)

	ddf_full = dd.concat([ddf_train, ddf_val])

	cat_cardinalities = {
		feat: len(EMBEDDING_MAPPINGS[feat])
		for feat in CATEGORICAL_FEATURES
	}

	model = EmbeddingMLP(
		cat_cardinalities=cat_cardinalities,
		cat_features=CATEGORICAL_FEATURES,
		num_cont_features=2 * len(NUMERICAL_FEATURES),
	).to(device)

	state_dict = torch.load(best_ckpt_path, map_location=device)
	model.load_state_dict(state_dict)

	criterion = nn.MSELoss()
	optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE * 0.3)  # LR más bajo para fine-tune
	scaler = torch.amp.GradScaler(device="cuda", enabled=AMP)

	FINE_TUNE_EPOCHS = 2  # por ejemplo

	for epoch in range(FINE_TUNE_EPOCHS):
		print(f"\n--- FINE-TUNE Epoch {epoch + 1}/{FINE_TUNE_EPOCHS} ---")
		model.train()

		partition_iterator = ddf_full.to_delayed()

		for partition_idx, partition in enumerate(partition_iterator):
			train_loss_sum = 0.0

			pdf = partition.compute()
			if pdf.empty:
				print(f"  Skipping empty partition {partition_idx+1}")
				continue

			pdf = process_partition(pdf)

			features_tensor = torch.tensor(
				pdf[CATEGORICAL_FEATURES
					+ NUMERICAL_FEATURES
					+ [f"{col}_is_missing" for col in NUMERICAL_FEATURES]
				].values,
				dtype=torch.float32,
			)
			target_tensor = torch.tensor(
				pdf[TARGET].values,
				dtype=torch.float32,
			)

			partition_dataset = TensorDataset(features_tensor, target_tensor)
			partition_loader = DataLoader(
				dataset=partition_dataset,
				batch_size=BATCH_SIZE,
				num_workers=0,
				pin_memory=True,
				shuffle=True,
			)

			for batch_idx, (data, target) in enumerate(partition_loader):
				data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)

				optimizer.zero_grad(set_to_none=True)
				with torch.amp.autocast(device_type="cuda", enabled=AMP):
					output = model(data)
					loss = criterion(output, target)

				if AMP:
					scaler.scale(loss).backward()
					scaler.step(optimizer)
					scaler.update()
				else:
					loss.backward()
					optimizer.step()

				train_loss_sum += loss.item()

			print(f"  Fine-tune Epoch {epoch+1} | Partition {partition_idx+1} | Loss: {train_loss_sum / len(partition_loader):.4f}")

	torch.save(model.state_dict(), "model_finetuned_train_val.pth")
	print("\n--- FINE-TUNE Finished ---")


In [13]:
fine_tune_with_val(best_ckpt_path="model_checkpoint_v1_1.pth")


--- FINE-TUNE Epoch 1/2 ---
  Fine-tune Epoch 1 | Partition 1 | Loss: 0.2329
  Fine-tune Epoch 1 | Partition 2 | Loss: 0.1928
  Fine-tune Epoch 1 | Partition 3 | Loss: 0.1800
  Fine-tune Epoch 1 | Partition 4 | Loss: 0.1844
  Fine-tune Epoch 1 | Partition 5 | Loss: 0.1992
  Fine-tune Epoch 1 | Partition 6 | Loss: 0.2009
  Fine-tune Epoch 1 | Partition 7 | Loss: 0.2181
  Fine-tune Epoch 1 | Partition 8 | Loss: 0.2184
  Fine-tune Epoch 1 | Partition 9 | Loss: 0.2137
  Fine-tune Epoch 1 | Partition 10 | Loss: 0.1873
  Fine-tune Epoch 1 | Partition 11 | Loss: 0.1721
  Fine-tune Epoch 1 | Partition 12 | Loss: 0.1577
  Fine-tune Epoch 1 | Partition 13 | Loss: 0.1610
  Fine-tune Epoch 1 | Partition 14 | Loss: 0.1808
  Fine-tune Epoch 1 | Partition 15 | Loss: 0.1942
  Fine-tune Epoch 1 | Partition 16 | Loss: 0.1958
  Fine-tune Epoch 1 | Partition 17 | Loss: 0.1901
  Fine-tune Epoch 1 | Partition 18 | Loss: 0.1562
  Fine-tune Epoch 1 | Partition 19 | Loss: 0.1536
  Fine-tune Epoch 1 | Partitio

In [10]:
class EmbeddingMLP(nn.Module):
	def _init_(
		self,
		cat_cardinalities: Dict[str, int],
		cat_features: List[str],
		num_cont_features: int,          # numéricas + indicadores de missing
		embedding_dim: int = 64,
		hidden_dims: List[int] = [1024, 512, 256, 128, 64],
		dropout: float = 0.1,
	):
		super()._init_()
		self.cat_features = cat_features
		self.num_cat = len(cat_features)
		self.num_cont_features = num_cont_features

		# Embeddings por feature categórica
		self.embeddings = nn.ModuleDict({
			feat: nn.Embedding(num_embeddings=cardinality, embedding_dim=embedding_dim)
			for feat, cardinality in cat_cardinalities.items()
		})

		# Dimensión de entrada al MLP: [contínuas + embeddings]
		total_cat_dim = embedding_dim * self.num_cat
		input_dim = self.num_cont_features + total_cat_dim

		layers = []
		prev_dim = input_dim
		for h in hidden_dims:
			layers.append(nn.Linear(prev_dim, h))
			layers.append(nn.ReLU())
			layers.append(nn.Dropout(dropout))
			prev_dim = h

		layers.append(nn.Linear(prev_dim, 1))  # regresión escalar

		self.mlp = nn.Sequential(*layers)

	def forward(self, x: torch.Tensor) -> torch.Tensor:
		"""
		x: [batch, num_cat + num_cont_features]
		columnas en el orden:
		[CATEGORICAL_FEATURES..., NUMERICAL_FEATURES..., NUMERICAL_IS_MISSING...]
		"""

		# 1) Separar categóricas y numéricas del gran vector
		x_cat_ids = x[:, :self.num_cat].long()     # [batch, num_cat]
		x_cont = x[:, self.num_cat:].float()       # [batch, num_cont_features]

		# 2) Embeddings categóricas
		emb_list = []
		for i, feat in enumerate(self.cat_features):
			emb = self.embeddings[feat](x_cat_ids[:, i])  # [batch, embedding_dim]
			emb_list.append(emb)

		if emb_list:
			x_cat_emb = torch.cat(emb_list, dim=1)        # [batch, embedding_dim * num_cat]
			x_input = torch.cat([x_cont, x_cat_emb], dim=1)
		else:
			x_input = x_cont

		out = self.mlp(x_input)   # [batch, 1]
		return out.squeeze(-1)

In [ ]:
DATASET_TEST_PATH = "./data/test"

ddf_test = dd.read_parquet(
	DATASET_TEST_PATH,
	engine="pyarrow",
	columns=[obj for obj in COLS_TO_READ if obj != "iap_revenue_d7"] + ["row_id"]
)

predictions = []
row_ids = []

partition_iterator = ddf_test.to_delayed()

for partition_idx, partition in enumerate(partition_iterator):

	print(f"Processing test partition {partition_idx+1}")

	test_pdf = partition.compute()


	row_ids.extend(test_pdf["row_id"].values)
	test_pdf = process_partition(test_pdf)

	test_features_tensor = torch.tensor(
		test_pdf[CATEGORICAL_FEATURES + NUMERICAL_FEATURES + [f"{col}_is_missing" for col in NUMERICAL_FEATURES]].values, 
		dtype=torch.float32
	)

	test_dataset = TensorDataset(test_features_tensor)
	test_loader = DataLoader(dataset=test_dataset, 
								batch_size=32768, 
								num_workers=0,
								pin_memory=True,
								shuffle=False)
	
	model = EmbeddingMLP(cat_cardinalities=cat_cardinalities, cat_features=CATEGORICAL_FEATURES, num_cont_features=2*len(NUMERICAL_FEATURES)).to(device)
	model.load_state_dict(torch.load("model_checkpoint_v4_1.pth"))
	model.eval()

	with torch.no_grad():
		for batch_idx, (data,) in enumerate(test_loader):
			data = data.to(device, non_blocking=True)

			output = model(data)

			predictions.extend(output.cpu().numpy())



Processing test partition 1


TypeError: EmbeddingMLP.__init__() got an unexpected keyword argument 'cat_cardinalities'

In [ ]:
pred_df = pd.DataFrame({
	"row_id": row_ids,
	"iap_revenue_d7": np.clip(np.expm1(predictions),a_min=0.0,a_max=10000.0)  # inverse of log1p
})
print(pred_df.head())
pred_df.to_csv("test_predictions.csv", index=False)
print("Test predictions saved to test_predictions.csv")

C:\Users\adria.flores\AppData\Local\Temp\ipykernel_21512\2318167038.py:3: RuntimeWarning: overflow encountered in expm1
  "iap_revenue_d7": np.maximum(np.expm1(predictions),0.0)  # inverse of log1p


                                 row_id  iap_revenue_d7
0  e2f514a9-d922-4a17-bf94-f228bf4cd82f    1.016614e-03
1  4bfc70d3-d619-410a-9683-4cd759f30f32    4.789741e-03
2  ad433b66-b41e-4157-a6fd-24cd30701f6a    2.974149e-08
3  5ed964d6-ddce-42e8-9fad-276eb7f64c2f    4.402779e-03
4  81b73a45-c395-4d08-a4a3-513873440db3    5.463762e-06
Test predictions saved to test_predictions.csv


In [11]:
DATASET_TEST_PATH = "./data/test"

ddf_test = dd.read_parquet(
	DATASET_TEST_PATH,
	engine="pyarrow",
	columns=[obj for obj in COLS_TO_READ if obj != "iap_revenue_d7"] + ["row_id"]
)

predictions = []
row_ids = []

partition_iterator = ddf_test.to_delayed()

for partition_idx, partition in enumerate(partition_iterator):

	print(f"Processing test partition {partition_idx+1}")

	test_pdf = partition.compute()


	row_ids.extend(test_pdf["row_id"].values)
	test_pdf = process_partition(test_pdf)

	test_features_tensor = torch.tensor(
		test_pdf[CATEGORICAL_FEATURES + NUMERICAL_FEATURES + [f"{col}_is_missing" for col in NUMERICAL_FEATURES]].values, 
		dtype=torch.float32
	)

	test_dataset = TensorDataset(test_features_tensor)
	test_loader = DataLoader(dataset=test_dataset, 
								batch_size=32768, 
								num_workers=0,
								pin_memory=True,
								shuffle=False)
	
	model = EmbeddingMLP(cat_cardinalities=cat_cardinalities, cat_features=CATEGORICAL_FEATURES, num_cont_features=2*len(NUMERICAL_FEATURES)).to(device)
	model.load_state_dict(torch.load("model_checkpoint_v3_5.pth"))
	model.eval()

	with torch.no_grad():
		for batch_idx, (data,) in enumerate(test_loader):
			data = data.to(device, non_blocking=True)

			output = model(data)

			predictions.extend(output.cpu().numpy())

predictions_exp = np.expm1(predictions)

pred_df = pd.DataFrame({
	"row_id": row_ids,
	"iap_revenue_d7": np.clip(predictions_exp, a_min=0.0, a_max=10000.0)
})
print(pred_df.head())
pred_df.to_csv("test_predictions.csv", index=False)
print("Test predictions saved to test_predictions.csv")

Processing test partition 1


TypeError: EmbeddingMLP.__init__() got an unexpected keyword argument 'cat_cardinalities'